# SH3GL2 - Single gene analysis in GP2 data for SAS population

## Description

Using "individual level data"

### 0. Getting Started

- Loading Python libraries
- Defining functions
- Installing packages

### 1. Copy data from workspace to cloud environment

### 2. Extract SH3GL2

### 3. Annotate SH3GL2 variants

### 4. Extract coding/non-syn variants

### 5. Calculate frequency in cases versus controls

### 6. Calculate frequency (homozygotes) in cases versus controls

### 7. Save out results

## Getting Started

### Loading Python libraries

### Defining functions

In [1]:
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd

# Numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery

#Import Sys
import sys as sys

In [2]:
# Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}', file=sys.stderr)
    !$command
    
def shell_return(command):
    print(f'Executing: {command}', file=sys.stderr)
    output = !$command
    return '\n'.join(output)

# Utility routine for printing a query before executing it
def bq_query(query):
    print(f'Executing: {query}', file=sys.stderr)
    return pd.read_gbq(query, project_id=BILLING_PROJECT_ID, dialect='standard')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)

### Set paths

In [3]:
# Set up billing project and data path variables
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

## Print the information to check we are in the proper release and billing 
## This will be different for you, the user, depending on the billing project your workspace is on
print('Billing and Workspace')
print(f'Workspace Name: {WORKSPACE_NAME}')
print(f'Billing Project: {BILLING_PROJECT_ID}')
print(f'Workspace Bucket, where you can upload and download data: {WORKSPACE_BUCKET}')
print('')


## GP2 v5.0
## Explicitly define release v5.0 path 
GP2_RELEASE_PATH = 'gs://gp2tier2/release5_11052023'
GP2_CLINICAL_RELEASE_PATH = f'{GP2_RELEASE_PATH}/clinical_data'
GP2_RAW_GENO_PATH = f'{GP2_RELEASE_PATH}/raw_genotypes'
GP2_IMPUTED_GENO_PATH = f'{GP2_RELEASE_PATH}/imputed_genotypes'
print('GP2 v5.0')
print(f'Path to GP2 v5.0 Clinical Data: {GP2_CLINICAL_RELEASE_PATH}')
print(f'Path to GP2 v5.0 Raw Genotype Data: {GP2_RAW_GENO_PATH}')
print(f'Path to GP2 v5.0 Imputed Genotype Data: {GP2_IMPUTED_GENO_PATH}')


## AMP-PD v3.0
## Explicitly define release v3.0 path 
AMP_RELEASE_PATH = 'gs://amp-pd-data/releases/2022_v3release_1115'
AMP_CLINICAL_RELEASE_PATH = f'{AMP_RELEASE_PATH}/clinical'

AMP_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS'
AMP_WGS_RELEASE_PLINK_PATH = os.path.join(AMP_WGS_RELEASE_PATH, 'plink')
AMP_WGS_RELEASE_PLINK_PFILES = os.path.join(AMP_WGS_RELEASE_PLINK_PATH, 'pfiles')

print('AMP-PD v3.0')
print(f'Path to AMP-PD v3.0 Clinical Data: {AMP_CLINICAL_RELEASE_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PATH}')
print(f'Path to AMP-PD v3.0 WGS Data: {AMP_WGS_RELEASE_PLINK_PFILES}')
print('')

Billing and Workspace
Workspace Name: Endophilin-A
Billing Project: terra-18d8e41c
Workspace Bucket, where you can upload and download data: gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4

GP2 v5.0
Path to GP2 v5.0 Clinical Data: gs://gp2tier2/release5_11052023/clinical_data
Path to GP2 v5.0 Raw Genotype Data: gs://gp2tier2/release5_11052023/raw_genotypes
Path to GP2 v5.0 Imputed Genotype Data: gs://gp2tier2/release5_11052023/imputed_genotypes
AMP-PD v3.0
Path to AMP-PD v3.0 Clinical Data: gs://amp-pd-data/releases/2022_v3release_1115/clinical
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink
Path to AMP-PD v3.0 WGS Data: gs://amp-pd-genomics/releases/2022_v3release_1115/wgs-WB-DWGS/plink/pfiles



### Install packages

#### Install Plink 1.9 and Plink 2.0

In [4]:
%%bash

mkdir -p ~/tools
cd ~/tools

if test -e /home/jupyter/tools/plink; then
echo "Plink1.9 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink \n    -------"
wget -N http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 
unzip -o plink_linux_x86_64_20190304.zip
echo -e "\n plink downloaded and unzipped in /home/jupyter/tools \n "

fi


if test -e /home/jupyter/tools/plink2; then
echo "Plink2 is already installed in /home/jupyter/tools/"

else
echo -e "Downloading plink2 \n    -------"
wget -N https://s3.amazonaws.com/plink2-assets/alpha3/plink2_linux_avx2_20220603.zip
unzip -o plink2_linux_avx2_20220603.zip
echo -e "\n plink2 downloaded and unzipped in /home/jupyter/tools \n "

fi

Plink1.9 is already installed in /home/jupyter/tools/
Plink2 is already installed in /home/jupyter/tools/


In [5]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Remote restrictions

In [6]:
%%bash

# chmod plink 1.9 
chmod u+x /home/jupyter/tools/plink

In [7]:
%%bash

# chmod plink 2.0
chmod u+x /home/jupyter/tools/plink2

### Install ANNOVAR

In [8]:
%%bash

# Install ANNOVAR:
# https://www.openbioinformatics.org/annovar/annovar_download_form.php

if test -e /home/jupyter/tools/annovar; then

echo "annovar is already installed in /home/jupyter/tools/"
else
echo "annovar is not installed"
cd /home/jupyter/tools/

wget http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is already installed in /home/jupyter/tools/


In [9]:
%%bash
ls /home/jupyter/tools/

annovar
annovar.latest.tar.gz
LICENSE
plink
plink2
plink2_linux_avx2_20220603.zip
plink_linux_x86_64_20190304.zip
prettify
toy.map
toy.ped


#### Install ANNOVAR: Download sources of annotation 

In [10]:
%%bash

cd /home/jupyter/tools/annovar/

perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20140902 humandb/
#perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
#perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_clinvar_20140902.txt.idx.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished d

In [11]:
%%bash
ls /home/jupyter/tools/annovar/

annotate_variation.pl
coding_change.pl
convert2annovar.pl
example
humandb
retrieve_seq_from_fasta.pl
table_annovar.pl
variants_reduction.pl


## Copy data from GP2 bucket to workspace

In [12]:
# Make a directory
print("Making a working directory")
WORK_DIR = f'/home/jupyter/SH3GL2_GP2/' 
shell_do(f'mkdir -p {WORK_DIR}') 

Making a working directory


Executing: mkdir -p /home/jupyter/SH3GL2_GP2/


In [13]:
# Check directory where GP2 data is
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_IMPUTED_GENO_PATH}')

Executing: gsutil -u terra-18d8e41c ls gs://gp2tier2/release5_11052023/imputed_genotypes


gs://gp2tier2/release5_11052023/imputed_genotypes/AAC/
gs://gp2tier2/release5_11052023/imputed_genotypes/AFR/
gs://gp2tier2/release5_11052023/imputed_genotypes/AJ/
gs://gp2tier2/release5_11052023/imputed_genotypes/AMR/
gs://gp2tier2/release5_11052023/imputed_genotypes/CAS/
gs://gp2tier2/release5_11052023/imputed_genotypes/EAS/
gs://gp2tier2/release5_11052023/imputed_genotypes/EUR/
gs://gp2tier2/release5_11052023/imputed_genotypes/FIN/
gs://gp2tier2/release5_11052023/imputed_genotypes/MDE/
gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/


In [14]:
#Copying data for SAS population 
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_IMPUTED_GENO_PATH}/SAS/chr9* {WORK_DIR}')

Executing: gsutil -u terra-18d8e41c -m cp -r gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/chr9* /home/jupyter/SH3GL2_GP2/


Copying gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/chr9_SAS_release5.log...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/chr9_SAS_release5.pgen...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/chr9_SAS_release5.pvar...
Copying gs://gp2tier2/release5_11052023/imputed_genotypes/SAS/chr9_SAS_release5.psam...
/ [4/4 files][149.2 MiB/149.2 MiB] 100% Done                                    
Operation completed over 4 objects/149.2 MiB.                                    


### Create a covariate file with GP2 data

In [15]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GP2_CLINICAL_RELEASE_PATH}')
shell_do(f'gsutil -u {BILLING_PROJECT_ID} -m cp -r {GP2_CLINICAL_RELEASE_PATH}/master_key_release5_final.csv {WORK_DIR}')

Executing: gsutil -u terra-18d8e41c ls gs://gp2tier2/release5_11052023/clinical_data


gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv
gs://gp2tier2/release5_11052023/clinical_data/release5_data_dictionary.csv


Executing: gsutil -u terra-18d8e41c -m cp -r gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv /home/jupyter/SH3GL2_GP2/


Copying gs://gp2tier2/release5_11052023/clinical_data/master_key_release5_final.csv...
/ [1/1 files][  2.5 MiB/  2.5 MiB] 100% Done                                    
Operation completed over 1 objects/2.5 MiB.                                      


In [16]:
cov = pd.read_csv(f'{WORK_DIR}/master_key_release5_final.csv')
cov.columns

Index(['GP2ID', 'GP2sampleID', 'manifest_id', 'phenotype', 'pheno_for_qc',
       'other_pheno', 'sex_for_qc', 'age', 'age_of_onset', 'age_at_diagnosis',
       'age_at_death', 'race_for_qc', 'family_history', 'region_for_qc',
       'study', 'pruned', 'pruned_reason', 'label', 'related'],
      dtype='object')

In [17]:
cov_reduced = cov[['GP2sampleID','GP2sampleID','sex_for_qc', 'age', 'phenotype']]
cov_reduced.columns = ['FID','IID', 'SEX','AGE','PHENO']
cov_reduced

,FID,IID,SEX,AGE,PHENO
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1,61.0,PD
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1,66.0,PD
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2,55.0,Control
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1,50.0,Control
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2,74.0,Control
...,...,...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1,NaN,Control
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2,NaN,Control
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1,NaN,Control
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2,NaN,Control


In [18]:
conditions = [
    (cov_reduced['PHENO'] == "PD"),
    (cov_reduced['PHENO'] == "Control")]

In [19]:
choices = [2,1]
cov_reduced['PHENOTYPE'] = np.select(conditions, choices, default=-9).astype(np.int64)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
cov_reduced

,FID,IID,SEX,AGE,PHENO,PHENOTYPE
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1,61.0,PD,2
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1,66.0,PD,2
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2,55.0,Control,1
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1,50.0,Control,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2,74.0,Control,1
...,...,...,...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1,NaN,Control,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2,NaN,Control,1
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1,NaN,Control,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2,NaN,Control,1


In [21]:
cov_reduced.reset_index(inplace=True)
cov_reduced.drop(columns=["index"], inplace=True)

/opt/conda/lib/python3.7/site-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [22]:
cov_reduced.drop(columns=['PHENO'], inplace=True)

In [23]:
cov_reduced

,FID,IID,SEX,AGE,PHENOTYPE
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1,61.0,2
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1,66.0,2
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2,55.0,1
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1,50.0,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2,74.0,1
...,...,...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1,NaN,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2,NaN,1
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1,NaN,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2,NaN,1


In [24]:
sex = cov_reduced[['FID','IID','SEX']]
sex

,FID,IID,SEX
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,1
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,1
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,2
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,2
...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,2
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,2


In [25]:
sex.to_csv(f'{WORK_DIR}/SEX.txt',index=False, sep="\t")

In [26]:
pheno = cov_reduced[['FID','IID','PHENOTYPE']]
pheno

,FID,IID,PHENOTYPE
0,IMMUNEPD_000001_s1,IMMUNEPD_000001_s1,2
1,IMMUNEPD_000002_s1,IMMUNEPD_000002_s1,2
2,IMMUNEPD_000003_s1,IMMUNEPD_000003_s1,1
3,IMMUNEPD_000004_s1,IMMUNEPD_000004_s1,1
4,IMMUNEPD_000005_s1,IMMUNEPD_000005_s1,1
...,...,...,...
26723,MDGAP-EBB_000149_s1,MDGAP-EBB_000149_s1,1
26724,MDGAP-EBB_000150_s1,MDGAP-EBB_000150_s1,1
26725,MDGAP-EBB_000151_s1,MDGAP-EBB_000151_s1,1
26726,MDGAP-EBB_000152_s1,MDGAP-EBB_000152_s1,1


In [27]:
pheno.to_csv(f'{WORK_DIR}/PHENO.txt',index=False, sep="\t")

## Extract SH3GL2 

In [28]:
%%bash
#For SAS
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# SH3GL2: gene positions on hg38 (from https://useast.ensembl.org/index.html)
/home/jupyter/tools/plink2 \
--pfile chr9_SAS_release5 \
--chr 9 \
--from-bp 17579066  \
--to-bp 17797124 \
--make-bed \
--out pheno_SH3GL2_SAS

PLINK v2.00a3.3LM AVX2 Intel (3 Jun 2022)      www.cog-genomics.org/plink/2.0/
(C) 2005-2022 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_SH3GL2_SAS.log.
Options in effect:
  --chr 9
  --from-bp 17579066
  --make-bed
  --out pheno_SH3GL2_SAS
  --pfile chr9_SAS_release5
  --to-bp 17797124

Start time: Mon Jul 10 14:25:26 2023
7450 MiB RAM detected; reserving 3725 MiB for main workspace.
Using up to 2 compute threads.
220 samples (98 females, 122 males; 220 founders) loaded from
chr9_SAS_release5.psam.
346642 variants loaded from chr9_SAS_release5.pvar.
1 binary phenotype loaded (80 cases, 135 controls).
1045 variants remaining after main filters.
Writing pheno_SH3GL2_SAS.fam ... done.
Writing pheno_SH3GL2_SAS.bim ... done.
Writing pheno_SH3GL2_SAS.bed ... 0%done.
End time: Mon Jul 10 14:25:26 2023


### Visualize plink files bim, fam and bed  and turn binary files to vcf

In [29]:
%%bash

# Visualize bim file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR


head pheno_SH3GL2_SAS.bim

9	chr9:17579372:T:C	0	17579372	C	T
9	chr9:17579692:T:G	0	17579692	G	T
9	chr9:17579727:T:C	0	17579727	C	T
9	chr9:17579763:C:T	0	17579763	T	C
9	chr9:17579765:C:G	0	17579765	G	C
9	chr9:17580294:G:T	0	17580294	T	G
9	chr9:17580618:A:T	0	17580618	T	A
9	chr9:17581041:G:A	0	17581041	A	G
9	chr9:17581152:G:A	0	17581152	A	G
9	chr9:17581170:C:G	0	17581170	G	C


In [30]:
%%bash

# Visualize fam file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_SAS.fam

#Count lines temp.fam file
wc -l pheno_SH3GL2_SAS.fam

0	APGS_000062_s1	0	0	1	2
0	APGS_000119_s1	0	0	1	2
0	APGS_000299_s1	0	0	1	2
0	APGS_000331_s1	0	0	1	2
0	APGS_000378_s1	0	0	2	2
0	APGS_000416_s1	0	0	1	2
0	APGS_000454_s1	0	0	2	2
0	APGS_000502_s1	0	0	2	2
0	APGS_000526_s1	0	0	1	2
0	APGS_000604_s1	0	0	2	2
220 pheno_SH3GL2_SAS.fam


In [31]:
%%bash

# Visualize bed file
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head pheno_SH3GL2_SAS.bed

l(��@ ��B� 8.� �,�>(8 ,��@"�L��� � @!��R�0,��+� ���<
(�ʿ<��(��"�*���
 *��*躠#ꂌ�"��.��������������������������������������������������������������������������������������ӿ�����w�����������������*�﫠����:�ú������?>�������������+���*��;ޮ~�����������������������������w����������������������߯����*�+���:ê�������?:�������x����+��쪾�;ު���������������������������������㿻����3���������������㢎�*�룠"±�
 (���<�?*���*���`2���
��
>�:��>�����*�﫠����:ú�뻮���?>�������x����+���*��;ޮ������*�+���:î�������?:�����������+���*��;ު������*�;���:ê�뻮���?:�������x����+���*��;�����㢞�*�룠ʲ�
 (���<�?���*���`2���
��
=�:��>��������������������������������㿻����3������������������*�﫠����:þ�������?>�����������+���*��;��������*�+���: î�������?:�������������+���*��;��������*�����zQþ�������?>�������������+���*��;���Я����*�;���:î�������?:�������������+���*��;���ЋC㢞�:Ȭ든"²jh
 (˞��<�>�|�*���`2���


In [32]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

# Turn binary files into VCF for SAS
/home/jupyter/tools/plink \
--bfile pheno_SH3GL2_SAS \
--recode vcf-fid \
--out pheno_SH3GL2_SAS

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pheno_SH3GL2_SAS.log.
Options in effect:
  --bfile pheno_SH3GL2_SAS
  --out pheno_SH3GL2_SAS
  --recode vcf-fid

7450 MB RAM detected; reserving 3725 MB for main workspace.
1045 variants loaded from .bim file.
220 people (122 males, 98 females) loaded from .fam.
215 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 220 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62%63%64%

## Annotate SH3GL2 variants

In [33]:
%%bash
#for SAS
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR
export PATH=$PATH:/home/jupyter/tools/rvtests/third/tabix-0.2.6/

### Bgzip and Tabix
bgzip pheno_SH3GL2_SAS.vcf

tabix -f -p vcf pheno_SH3GL2_SAS.vcf.gz

[bgzip] can't create pheno_SH3GL2_SAS.vcf.gz: File exists


In [34]:
%%bash
#For SAS
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

### Annotate variants using ANNOVAR: https://annovar.openbioinformatics.org/en/latest/ 
perl /home/jupyter/tools/annovar/table_annovar.pl pheno_SH3GL2_SAS.vcf.gz /home/jupyter/tools/annovar/humandb/ -buildver hg38 \
-out pheno_SH3GL2_SAS.annovar \
-remove -protocol refGene,clinvar_20140902 \
-operation g,f \
--nopolish \
-nastring . \
-vcfinput


NOTICE: Running with system command <convert2annovar.pl  -includeinfo -allsample -withfreq -format vcf4 pheno_SH3GL2_SAS.vcf.gz > pheno_SH3GL2_SAS.annovar.avinput>
NOTICE: Finished reading 1052 lines from VCF file
NOTICE: A total of 1045 locus in VCF file passed QC threshold, representing 998 SNPs (639 transitions and 359 transversions) and 47 indels/substitutions
NOTICE: Finished writing allele frequencies based on 219560 SNP genotypes (140580 transitions and 78980 transversions) and 10340 indels/substitutions for 220 samples

NOTICE: Running with system command </home/jupyter/tools/annovar/table_annovar.pl pheno_SH3GL2_SAS.annovar.avinput /home/jupyter/tools/annovar/humandb/ -buildver hg38 -outfile pheno_SH3GL2_SAS.annovar -remove -protocol refGene,clinvar_20140902 -operation g,f --nopolish -nastring . -otherinfo>
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.

## Extract coding and non-syn variants

In [35]:
# Visualize multianno file for SAS
SH3GL2_SAS = pd.read_csv(f'{WORK_DIR}/pheno_SH3GL2_SAS.annovar.hg38_multianno.txt', sep = '\t')
SH3GL2_SAS.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo223,Otherinfo224,Otherinfo225,Otherinfo226,Otherinfo227,Otherinfo228,Otherinfo229,Otherinfo230,Otherinfo231,Otherinfo232
0,9,17579372,17579372,C,T,intronic,SH3GL2,.,.,.,...,./.,0/0,0/0,0/0,./.,0/1,0/1,0/0,./.,./.
1,9,17579692,17579692,G,T,intronic,SH3GL2,.,.,.,...,0/1,0/1,0/1,0/0,0/0,0/1,0/1,1/1,0/1,0/0
2,9,17579727,17579727,T,C,intronic,SH3GL2,.,.,.,...,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,./.,0/0
3,9,17579763,17579763,C,T,intronic,SH3GL2,.,.,.,...,0/0,0/0,0/0,0/0,0/0,0/1,0/0,0/0,0/0,./.
4,9,17579765,17579765,C,G,intronic,SH3GL2,.,.,.,...,0/1,0/1,0/1,0/0,0/0,./.,0/1,./.,0/1,0/0


In [36]:
SH3GL2_SAS.count()

Chr             1045
Start           1045
End             1045
Ref             1045
Alt             1045
                ... 
Otherinfo228    1045
Otherinfo229    1045
Otherinfo230    1045
Otherinfo231    1045
Otherinfo232    1045
Length: 243, dtype: int64

In [37]:
# Filter exonic variants for SAS
coding_SAS = SH3GL2_SAS[SH3GL2_SAS['Func.refGene'] == 'exonic']
coding_SAS.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo223,Otherinfo224,Otherinfo225,Otherinfo226,Otherinfo227,Otherinfo228,Otherinfo229,Otherinfo230,Otherinfo231,Otherinfo232


In [38]:
#Filter exonic and syn vars in SAS
coding_synonymous_SAS = SH3GL2_SAS[(SH3GL2_SAS['Func.refGene'] == 'exonic') & (SH3GL2_SAS['ExonicFunc.refGene'] == 'synonymous SNV')]
coding_synonymous_SAS.count()

Chr             0
Start           0
End             0
Ref             0
Alt             0
               ..
Otherinfo228    0
Otherinfo229    0
Otherinfo230    0
Otherinfo231    0
Otherinfo232    0
Length: 243, dtype: int64

In [39]:
#Filter exonic and non-syn vars in SAS
coding_nonsynonymous_SAS = SH3GL2_SAS[(SH3GL2_SAS['Func.refGene'] == 'exonic') & (SH3GL2_SAS['ExonicFunc.refGene'] == 'nonsynonymous SNV')]
coding_nonsynonymous_SAS.count()

Chr             0
Start           0
End             0
Ref             0
Alt             0
               ..
Otherinfo228    0
Otherinfo229    0
Otherinfo230    0
Otherinfo231    0
Otherinfo232    0
Length: 243, dtype: int64

In [40]:
coding_nonsynonymous_SAS.to_csv(f'{WORK_DIR}/coding_nonsynonymous.txt', sep = '\t', index = False)
coding_nonsynonymous_SAS.head()

,Chr,Start,End,Ref,Alt,Func.refGene,Gene.refGene,GeneDetail.refGene,ExonicFunc.refGene,AAChange.refGene,...,Otherinfo223,Otherinfo224,Otherinfo225,Otherinfo226,Otherinfo227,Otherinfo228,Otherinfo229,Otherinfo230,Otherinfo231,Otherinfo232


In [41]:
# Filter intronic variants in SAS
intronic_SAS = SH3GL2_SAS[SH3GL2_SAS['Func.refGene'] == 'intronic']
intronic_SAS.count()

Chr             1042
Start           1042
End             1042
Ref             1042
Alt             1042
                ... 
Otherinfo228    1042
Otherinfo229    1042
Otherinfo230    1042
Otherinfo231    1042
Otherinfo232    1042
Length: 243, dtype: int64

In [42]:
# Filter UTR3 variants in SAS
UTR3_SAS= SH3GL2_SAS[SH3GL2_SAS['Func.refGene'] == ('UTR3')]
UTR3_SAS.count()

Chr             3
Start           3
End             3
Ref             3
Alt             3
               ..
Otherinfo228    3
Otherinfo229    3
Otherinfo230    3
Otherinfo231    3
Otherinfo232    3
Length: 243, dtype: int64

In [43]:
# Filter UTR5 variants in SAS
UTR5_SAS = SH3GL2_SAS[SH3GL2_SAS['Func.refGene'] == ('UTR5')]
UTR5_SAS.count()

Chr             0
Start           0
End             0
Ref             0
Alt             0
               ..
Otherinfo228    0
Otherinfo229    0
Otherinfo230    0
Otherinfo231    0
Otherinfo232    0
Length: 243, dtype: int64

In [44]:
# Calculate freq - cases vs controls for SAS

In [45]:
reduced_coding_nonsynonymous_SAS = coding_nonsynonymous_SAS[["Chr", "Start", "End", "Gene.refGene"]]
reduced_coding_nonsynonymous_SAS.to_csv(f'{WORK_DIR}/reduced_coding_nonsynonymous_SAS.txt', sep = '\t', index = False, header= False)

In [46]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

head reduced_coding_nonsynonymous_SAS.txt

In [47]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_SAS --extract range reduced_coding_nonsynonymous_SAS.txt --assoc --out coding_nonsynonymous_pheno_SH3GL2_SAS --allow-no-sex

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_SAS.log.
Options in effect:
  --allow-no-sex
  --assoc
  --bfile pheno_SH3GL2_SAS
  --extract range reduced_coding_nonsynonymous_SAS.txt
  --out coding_nonsynonymous_pheno_SH3GL2_SAS

7450 MB RAM detected; reserving 3725 MB for main workspace.
1045 variants loaded from .bim file.
220 people (122 males, 98 females) loaded from .fam.
215 phenotype values loaded from .fam.


Error: All variants excluded by '--extract range'.


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/SH3GL2_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile pheno_SH3GL2_SAS --extract range reduced_coding_nonsynonymous_SAS.txt --assoc --out coding_nonsynonymous_pheno_SH3GL2_SAS --allow-no-sex\n"' returned non-zero exit status 13.

In [ ]:
%%bash

#Bonferroni correction
WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_SAS --extract range reduced_coding_nonsynonymous_SAS.txt --make-bed --max-maf 0.05 --out coding_nonsynonymous_SAS_rare_0.05 --allow-no-sex

In [ ]:
SH3GL2_SAS_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_SH3GL2_SAS.assoc', delim_whitespace=True)
SH3GL2_SAS_freq.head(10)

In [ ]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile pheno_SH3GL2_SAS --extract range reduced_coding_nonsynonymous_SAS.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_SAS

In [48]:
#Some errors may appear if no variants are found
SH3GL2_SAS_freq = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_SH3GL2_SAS.assoc', delim_whitespace=True)
SH3GL2_SAS_freq.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/SH3GL2_GP2//coding_nonsynonymous_pheno_SH3GL2_SAS.assoc'

In [49]:
SH3GL2_SAS_recode = pd.read_csv(f'{WORK_DIR}/coding_nonsynonymous_pheno_SH3GL2_SAS.raw', delim_whitespace=True)
SH3GL2_SAS_recode.head()

FileNotFoundError: [Errno 2] No such file or directory: '/home/jupyter/SH3GL2_GP2//coding_nonsynonymous_pheno_SH3GL2_SAS.raw'

In [50]:
%%bash

WORK_DIR='/home/jupyter/SH3GL2_GP2/'
cd $WORK_DIR

/home/jupyter/tools/plink --bfile  pheno_SH3GL2_SAS --extract range reduced_coding_nonsynonymous_SAS.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_SAS

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to coding_nonsynonymous_pheno_SH3GL2_SAS.log.
Options in effect:
  --bfile pheno_SH3GL2_SAS
  --extract range reduced_coding_nonsynonymous_SAS.txt
  --out coding_nonsynonymous_pheno_SH3GL2_SAS
  --recode A

7450 MB RAM detected; reserving 3725 MB for main workspace.
1045 variants loaded from .bim file.
220 people (122 males, 98 females) loaded from .fam.
215 phenotype values loaded from .fam.


Error: All variants excluded by '--extract range'.


CalledProcessError: Command 'b"\nWORK_DIR='/home/jupyter/SH3GL2_GP2/'\ncd $WORK_DIR\n\n/home/jupyter/tools/plink --bfile  pheno_SH3GL2_SAS --extract range reduced_coding_nonsynonymous_SAS.txt --recode A --out coding_nonsynonymous_pheno_SH3GL2_SAS\n"' returned non-zero exit status 13.

In [51]:
# Explore HMZs for chr9: in cases
SH3GL2_hom_cases_SAS = SH3GL2_SAS_recode[(SH3GL2_SAS_recode['chr9:'] == 2) & (SH3GL2_SAS_recode['PHENOTYPE'] == 2)]
SH3GL2_hom_cases_SAS.head()

NameError: name 'SH3GL2_SAS_recode' is not defined

In [52]:
SH3GL2_hom_cases_SAS.shape

NameError: name 'SH3GL2_hom_cases_SAS' is not defined

In [53]:
# Explore HMZs for chr9: in controls
SH3GL2_hom_controls_SAS = SH3GL2_SAS_recode[(SH3GL2_SAS_recode['chr9:'] == 2) & (SH3GL2_SAS_recode['PHENOTYPE'] == 1)]
SH3GL2_hom_controls_SAS.head()

NameError: name 'SH3GL2_SAS_recode' is not defined

In [54]:
SH3GL2_hom_controls_SAS.shape

NameError: name 'SH3GL2_hom_controls_SAS' is not defined

## Save out results..!

In [55]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp -r {WORK_DIR} {WORKSPACE_BUCKET}')

Executing: gsutil -mu terra-18d8e41c cp -r /home/jupyter/SH3GL2_GP2/ gs://fc-f7a400c1-827e-48f8-b7b6-90c488a000a4


Copying file:///home/jupyter/SH3GL2_GP2/coding_nonsynonymous_EAS_rare_0.05.log [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_EUR.annovar.avinput [Content-Type=application/octet-stream]...
Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_AJ.annovar.hg38_multianno.vcf [Content-Type=text/vcard]...
Copying file:///home/jupyter/SH3GL2_GP2/pheno_SH3GL2_EUR.vcf.gz [Content-Type=text/vcard]...
Copying file:///home/jupyter/SH3GL2_GP2/chr9_AJ_release5.pgen [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run          
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads su